In [1]:
###DEBUG###
import os

os.environ['API_KEY'] = "889baad2-02b0-4c78-abe1-1952700d33ef" 
os.environ['NONINTERACT'] = 'True'

In [2]:
import json
import math
import os
import random
import sys
import time
import gc

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
import random

import qnt.ta as qnta
import qnt.data as qndata
import qnt.backtester as qnbt

import qnt.stats as qnstats
import qnt.output as output
import qnt.graph as qngraph

NOTICE: The environment variable DATA_BASE_URL was not specified. The default value is 'https://data-api.quantiacs.io/'
NOTICE: The environment variable CACHE_RETENTION was not specified. The default value is '7'
NOTICE: The environment variable CACHE_DIR was not specified. The default value is 'data-cache'


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
import os
for asset in ['F_AD', 'F_AE', 'F_AX', 'F_BC', 'F_BG', 'F_BP', 'F_C', 'F_CA',
                  'F_CC', 'F_CD', 'F_CF', 'F_CT', 'F_DM', 'F_DT', 'F_DX', 'F_EB',
                  'F_EC', 'F_ES', 'F_F', 'F_FB', 'F_FP', 'F_FY', 'F_GC', 'F_GS',
                  'F_GX', 'F_JY', 'F_KC', 'F_LX', 'F_MD', 'F_ND', 'F_NG', 'F_NH',
                  'F_NQ', 'F_NY', 'F_OJ', 'F_QT', 'F_RB', 'F_RF', 'F_RP', 'F_RU',
                  'F_RY', 'F_SB', 'F_SF', 'F_SI', 'F_SS', 'F_SX', 'F_TU', 'F_TY',
                  'F_UB', 'F_UZ', 'F_VX', 'F_W', 'F_XX', 'F_YM']:
    try:
        filename = asset+'checkpoint.pt'
        os.remove(filename)
    except: 
        pass
    
os.remove('generalcheckpoint.pt')
os.remove('fractions.nc.gz')

In [6]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

class ElecDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item, label

In [7]:
def load_data():
    data = qndata.futures.load_data(min_date = "2002-01-01", 
                                    max_date = "2005-12-31", 
                                    #tail=period,
                                   )
    return data

data = load_data()
close = data.sel(field="close").ffill('time').bfill('time')#.fillna(0)
asset_name_all = data.coords['asset'].values
print(list(asset_name_all))
print(len(asset_name_all))

['F_AD', 'F_AE', 'F_AG', 'F_AH', 'F_AX', 'F_BC', 'F_BG', 'F_BO', 'F_BP', 'F_C', 'F_CA', 'F_CC', 'F_CD', 'F_CF', 'F_CL', 'F_CT', 'F_DE', 'F_DM', 'F_DT', 'F_DX', 'F_EB', 'F_EC', 'F_ED', 'F_ES', 'F_F', 'F_FB', 'F_FP', 'F_FV', 'F_FY', 'F_GC', 'F_GS', 'F_GX', 'F_HG', 'F_HO', 'F_JY', 'F_KC', 'F_LR', 'F_LX', 'F_MD', 'F_MP', 'F_ND', 'F_NG', 'F_NH', 'F_NQ', 'F_NY', 'F_OJ', 'F_PA', 'F_PL', 'F_QT', 'F_RB', 'F_RF', 'F_RP', 'F_RR', 'F_RU', 'F_RY', 'F_S', 'F_SB', 'F_SF', 'F_SI', 'F_SS', 'F_SX', 'F_TR', 'F_TU', 'F_TY', 'F_UB', 'F_US', 'F_UZ', 'F_VX', 'F_W', 'F_XX', 'F_YM']
71


In [8]:
len(['F_AD', 'F_AE', 'F_AX', 'F_BC', 'F_BG', 'F_BP',
       'F_C', 'F_CA', 'F_CC', 'F_CD', 'F_CF', 'F_CT', 'F_DM',
       'F_DT', 'F_DX', 'F_EC', 'F_ES', 'F_FB', 'F_FP',
       'F_FY', 'F_GC', 'F_GS', 'F_GX', 'F_JY', 'F_KC',
       'F_LX', 'F_MD', 'F_ND', 'F_NG', 'F_NH', 'F_NQ', 'F_NY',
       'F_OJ', 'F_QT', 'F_RF', 'F_RP', 'F_RU',
       'F_RY', 'F_SB', 'F_SF', 'F_SI',
           'F_TY', 'F_UZ', 'F_VX', 'F_W', 'F_YM'])

46

In [9]:
len(['F_AD', 'F_AE', 'F_AX', 'F_BC', 'F_BG', 'F_BP', 'F_C', 'F_CA', 'F_CC', 'F_CD', 'F_CF', 'F_CT', 'F_DM', 'F_DT', 'F_DX', 'F_EB', 'F_EC', 'F_ES', 'F_F', 'F_FB', 'F_FP', 'F_FY', 'F_GC', 'F_GS', 'F_GX', 'F_JY', 'F_KC', 'F_LX', 'F_MD', 'F_ND', 'F_NG', 'F_NH', 'F_NQ', 'F_NY', 'F_OJ', 'F_QT', 'F_RB', 'F_RF', 'F_RP', 'F_RU', 'F_RY', 'F_SB', 'F_SF', 'F_SI', 'F_SS', 'F_SX', 'F_TU', 'F_TY', 'F_UB', 'F_UZ', 'F_VX', 'F_W', 'F_XX', 'F_YM'])

54

In [10]:
in_seq1 = []

for asset in ["F_C", "F_DM", "F_DT", "F_EB", "F_SS", "F_UB"]:
#                 ['F_AD', 'F_AE', 'F_AX', 'F_BC', 'F_BG', 'F_BP', 'F_C', 'F_CA',
#                   'F_CC', 'F_CD', 'F_CF', 'F_CT', 'F_DM', 'F_DT', 'F_DX', 'F_EB',
#                   'F_EC', 'F_ES', 'F_F', 'F_FB', 'F_FP', 'F_FY', 'F_GC', 'F_GS',
#                   'F_GX', 'F_JY', 'F_KC', 'F_LX', 'F_MD', 'F_ND', 'F_NG', 'F_NH',
#                   'F_NQ', 'F_NY', 'F_OJ', 'F_QT', 'F_RB', 'F_RF', 'F_RP', 'F_RU',
#                   'F_RY', 'F_SB', 'F_SF', 'F_SI', 'F_SS', 'F_SX', 'F_TU', 'F_TY',
#                   'F_UB', 'F_UZ', 'F_VX', 'F_W', 'F_XX', 'F_YM']:
    single_close = close.sel(asset=[asset]).to_pandas()
    #print(single_close)
    close_ret = np.log(single_close/single_close.shift(1)).dropna()
    close_arr = np.array(close_ret)
    in_seq1.extend(close_arr)

In [11]:
#scaling 
from sklearn.preprocessing import MinMaxScaler
scaler_close = MinMaxScaler()
close_scaled = scaler_close.fit_transform(in_seq1)

In [12]:
# choose a number of time steps
n_steps = 50
# split into samples
X, y = split_sequence(close_scaled, n_steps)
y_reverse = scaler_close.inverse_transform(y)
y = np.where(y_reverse < 0, -1, 1 )

In [13]:
split = int(len(X)*0.9)
train_x = np.array(X[:split])
valid_x = np.array(X[split:])
train_y = y[:split]
valid_y = y[split:]

train = ElecDataset(train_x.reshape(train_x.shape[0], 1, train_x.shape[1]),train_y)
valid = ElecDataset(valid_x.reshape(valid_x.shape[0], 1, valid_x.shape[1]),valid_y)
train_loader = torch.utils.data.DataLoader(train,batch_size=64,shuffle=True, drop_last = True)
valid_loader = torch.utils.data.DataLoader(valid,batch_size=64,shuffle=True, drop_last = True)

CNN-1D Model

In [14]:
class TS(nn.Module):
    def __init__(self, n_input= 1, n_output=1):
        super(TS, self).__init__()

        self.conv1 = nn.Conv1d(1, 1, kernel_size = 3)
        self.conv2 = nn.Conv1d(1, 1, kernel_size = 3)
        self.pool1d = nn.MaxPool1d(2,2)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(11,4)
        self.fc2 = nn.Linear(4, n_output)
        #self.double()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool1d(x)
        # x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


model = TS(n_input=1, n_output=1)
model.to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n = count_parameters(model)
print("Number of parameters: %s" % n)

TS(
  (conv1): Conv1d(1, 1, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(1, 1, kernel_size=(3,), stride=(1,))
  (pool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU(inplace=True)
  (fc1): Linear(in_features=11, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)
Number of parameters: 61


In [15]:
train_losses = []
valid_losses = []

def Train():
    
    running_loss = .0
    
    model.train()
    
    for idx, (inputs, labels) in enumerate(train_loader):
        labels = labels.unsqueeze(1)
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())
        loss = criterion(preds,labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.cpu().detach().numpy())
    
    print(f'train_loss {train_loss}')

def Valid():
    running_loss = .0
    
    model.eval()
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_loader):
            labels = labels.unsqueeze(1)
            inputs = inputs.to(device)
            #print(inputs)
            labels = labels.to(device)
            #print(labels)
            optimizer.zero_grad()
            preds = model(inputs.float())
            #print(preds)
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(valid_loader)
        valid_losses.append(valid_loss.cpu().detach().numpy())
        print(f'valid_loss {valid_loss}')

In [16]:
epochs = 10
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()
    Valid()
    gc.collect()

epochs 1/10
train_loss 1.2850134372711182
valid_loss 1.0796388387680054
epochs 2/10
train_loss 1.0989995002746582
valid_loss 1.0039914846420288
epochs 3/10
train_loss 0.9614925980567932
valid_loss 0.9977400302886963
epochs 4/10
train_loss 0.8929240703582764
valid_loss 1.0338718891143799
epochs 5/10
train_loss 0.8729823231697083
valid_loss 1.0511887073516846
epochs 6/10
train_loss 0.8693141937255859
valid_loss 1.0589189529418945
epochs 7/10
train_loss 0.8690187335014343
valid_loss 1.052855134010315
epochs 8/10
train_loss 0.8689975738525391
valid_loss 1.0773388147354126
epochs 9/10
train_loss 0.8692571520805359
valid_loss 1.0736440420150757
epochs 10/10
train_loss 0.8692848086357117
valid_loss 1.077738642692566


In [17]:
def save_ckp(state, checkpoint_dir):
        f_path = checkpoint_dir
        torch.save(state, f_path)

checkpoint = {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
save_path = 'generalcheckpoint.pt'
save_ckp(checkpoint, save_path)

In [18]:
# https://quantiacs.com/documentation/en/examples/machine_learning_ridgeclassifier.html

import xarray as xr

import qnt.backtester as qnbt
import qnt.data as qndata
import numpy as np
import logging
import torch

def load_data(period):
    return qndata.futures.load_data(
        assets = ["F_C", "F_DM", "F_DT", "F_EB", "F_SS", "F_UB"],
        tail=period)

def predict_weights(market_data):
    
    def save_ckp(state, checkpoint_dir):
        f_path = checkpoint_dir
        torch.save(state, f_path)
  
    def load_ckp(checkpoint_fpath, model, optimizer):
        checkpoint = torch.load(checkpoint_fpath)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer, checkpoint['epoch']
        
    def define_model():
        model = TS(n_input=1, n_output=1)
        return model

    def Train():
        running_loss = .0
        model.train()
        
        for idx, (inputs, labels) in enumerate(train_loader):
            labels = labels.unsqueeze(1)
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())
            loss = criterion(preds,labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss
            
        train_loss = running_loss/len(train_loader)
        train_losses.append(train_loss.cpu().detach().numpy())
        #print(f'train_loss {train_loss}')

    data = market_data.copy(True)
    close = data.sel(field="close").ffill('time').bfill('time').fillna(0).to_pandas()
    asset_name_all = data.coords['asset'].values
    predict_weights_next_day_df = data.sel(field="close").isel(time=-1).to_pandas()

    for asset_name in asset_name_all:
        close_returns = np.log(close[asset_name]/close[asset_name].shift(1)).dropna()
        close_array = np.array(close_returns)
        in_seq1 = close_array[:-1]
        
        #scaling
        close_scaled = scaler_close.transform(in_seq1.reshape(-1, 1))
        # choose a number of time steps
        n_steps = 50
        # split into samples
        X, y = split_sequence(close_scaled, n_steps)
        y_reverse = scaler_close.inverse_transform(y)
        y = np.where(y_reverse < 0, -1, 1 )

        # dataloader
        train = ElecDataset(X.reshape(X.shape[0],1, X.shape[1])[-5:],y[-5:])
        train_loader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=True)
        
        # model and optimizer
        model = define_model()
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
        criterion = nn.MSELoss()

        try:
            # load from 
            try:
                ckp_path = asset_name+'checkpoint.pt'
                model, optimizer, start_epoch = load_ckp(ckp_path, model, optimizer)
                #print("loading asset model")
            except: 
                ckp_path = 'generalcheckpoint.pt'
                model, optimizer, start_epoch = load_ckp(ckp_path, model, optimizer)
                #print("loading general model") 

            # train the model with current data
            epochs = 1
            train_losses = []
            for epoch in range(epochs):
                #print('epochs {}/{}'.format(epoch+1,epochs))
                Train()
                gc.collect()

            # save the model
            checkpoint = {
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            save_path = asset_name+'checkpoint.pt'
            save_ckp(checkpoint, save_path)
            
            x = close_scaled[-n_steps:]
            inputs = torch.from_numpy(x.reshape(1, 1, n_steps))
            
            model.eval()
            with torch.no_grad():
                inputs = inputs.to(device)
                optimizer.zero_grad()
                preds = model(inputs.float())
           
            predict_weights_next_day_df[asset_name] = preds[0]
            
        except:
            logging.exception("model failed")
            # if there is exception, return zero values
            return xr.zeros_like(data.isel(field=0, time=0))

    #print(predict_weights_next_day_df)
    return predict_weights_next_day_df.to_xarray()


weights = qnbt.backtest(
    competition_type="futures",
    load_data=load_data,
    lookback_period=100,
    start_date='2006-01-01',
    #test_period = 3,
    strategy=predict_weights,
    analyze=True,
    build_plots=True
)

Run last pass...
Load data...
Run strategy...
Load data for cleanup...
Output cleaning...
fix uniq
Normalization...
Output cleaning is complete.
Write result...


NOTICE: The environment variable OUTPUT_PATH was not specified. The default value is 'fractions.nc.gz'


Write output: fractions.nc.gz
---
Run first pass...
Load data...
Run strategy...
---
Load full data...
---
Run iterations...



100% (3992 of 3992) |####################| Elapsed Time: 0:51:26 Time:  0:51:26


Merge outputs...
Load data for cleanup and analysis...
Output cleaning...
fix uniq
ffill if the current price is None...
Check missed dates...
WARNING! Output contain missed dates.
Adding missed dates and set zero...
Normalization...
Output cleaning is complete.
Write result...


...

---
Align...
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Output:


asset,F_AD,F_AE,F_AG,F_AH,F_AX,F_BC,F_BG,F_BO,F_BP,F_C
time,,,,,,,,,,
2021-07-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.135761
2021-07-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132300
2021-07-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130345
2021-07-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128775
2021-07-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.127925
2021-07-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.127765
2021-07-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.127731
2021-07-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128229
2021-07-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128378


Stats:


field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2021-07-12,1.866315,0.001410,0.040615,-0.019079,-0.095576,1.009973,0.041020,0.978934,6.0,0.028231,45.580729
2021-07-13,1.869179,0.001535,0.040612,-0.017573,-0.095576,1.012338,0.041113,0.984887,6.0,0.028228,45.581241
2021-07-14,1.874137,0.002653,0.040612,-0.014967,-0.095576,1.016458,0.041280,0.993268,6.0,0.028226,45.581999
2021-07-15,1.872680,-0.000778,0.040607,-0.015733,-0.095576,1.015026,0.041217,1.000000,6.0,0.028222,45.582158
2021-07-16,1.875589,0.001554,0.040604,-0.014204,-0.095576,1.017420,0.041311,1.000000,6.0,0.028219,45.582581
2021-07-19,1.871321,-0.002276,0.040603,-0.016448,-0.095576,1.013412,0.041148,1.000000,6.0,0.028217,45.584734
2021-07-20,1.876651,0.002849,0.040604,-0.013646,-0.095576,1.017832,0.041328,1.000000,6.0,0.028215,45.587391
2021-07-21,1.879740,0.001646,0.040601,-0.012023,-0.095576,1.020370,0.041428,1.000000,6.0,0.028215,45.590645
2021-07-22,1.879679,-0.000032,0.040596,-0.012055,-0.095576,1.020185,0.041415,1.000000,6.0,0.028216,45.594240


---


1. 2020, input_dim = 50, 1 epoch
sr = 1.47
2. 2013, input_dim = 50, 1 epoch
sr = 0.587
3. 2019, input_dim = 50, 1 epoch
sr = 1.12 
4. 2019, input dim = 50, 2 epochs
sr = 1.32
5. 2019, input dim = 40, 2 epochs
sr = 1.49
6. 2019, input dim = 30, 2 epochs
sr = 1.33
7. 2017, input dim = 40, 2 epochs
sr = 0.61
8. 2017, input dim = 40, 1 epochs
sr = 0.59
9. 2019, input dim = 40, 1 epochs
sr = 1.12
10. 2006, input dim = 40, 1 epoch (13 assets)
sr = 0.55
10. 2006, input dim = 60, 1 epoch (10 assets)
sr = 0.53

F_EB, F_SS: sr = 0.8

F_AD, F_EB, F_SS: sr = 0.4?? forgot

F_EB, F_SS, F_W: sr = 0.64

["F_C", "F_EB", "F_SS", "F_W"]: sr = 0.64

["F_C", "F_EB", "F_SS"]: sr = 0.78

["F_C", "F_DT", "F_EB", "F_SS"]: sr = 0.88

["F_C", "F_DT", "F_EB", "F_F", "F_SS"]: sr = 0.81

["F_C", "F_DT", "F_EB", "F_SS", "F_UB"]: sr = 0.93

["F_C", "F_DT", "F_EB", "F_NG", "F_SS", "F_UB"]: sr = 0.73

["F_C", "F_DT", "F_EB", "F_NQ", "F_SS", "F_UB"]: sr = 0.87

["F_C", "F_DT", "F_EB", "F_SS", "F_UB", "F_UZ"]: sr = 0.88

["F_C", "F_DT", "F_EB", "F_SS", "F_W", "F_UB"]: sr = 0.71